In [3]:
import numpy as np
import pandas as pd
from timeit import Timer
from collections import defaultdict
from IPython.core.display import display, HTML

### Question 3:

In [13]:
fields = ['SEQN', 'RIDAGEYR', 'RIAGENDR', 'RIDRETH3','DMDEDUC2','DMDMARTL','RIDSTATR', 
          'SDMVPSU', 'SDMVSTRA', 'WTMEC2YR', 'WTINT2YR' ]

In [17]:
# Read the dataset with the specified fields
df1112 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/DEMO_G.XPT')[fields]
df1112['cohort'] = '2011-2012'
df1314 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/DEMO_H.XPT')[fields]
df1314['cohort'] = '2013-2014'
df1516 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.XPT')[fields]
df1516['cohort'] = '2015-2016'

df1718 = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT')[fields]
df1718['cohort'] = '2017-2018'
Demo = df1112.append(df1314).append(df1516).append(df1718)

In [18]:
# Rename variables
Demo = Demo.rename({'SEQN' : 'id' , 'RIDAGEYR' : 'age',
            'RIAGENDR' : 'gender',
            'RIDRETH3' : 'race',
            'DMDEDUC2' : 'education','DMDMARTL' : 'marital_status',
            'RIDSTATR' : 'interview_exam_status', 'SDMVPSU' : 'psu', 
            'SDMVSTRA' : 'stratum', 'WTMEC2YR' : 'two_year_mec_wt' , 
            'WTINT2YR' : 'twoyr_interview_wt'}, axis = 1)
# Changed variables for appropriate category names
Demo['gender'] = pd.Categorical(Demo['gender'].replace( {1: 'Male', 2: 'Female'}))
Demo['race'] = pd.Categorical(Demo['race'].replace(
                            {1: 'Mexican American', 
                            2: 'Other Hispanic', 
                            3: 'Non-Hispanic White',
                            4: 'Non-Hispanic Black',
                            6: 'Non-Hispanic Asian',
                            7: 'Other Race - Including Multi-Racial',
                            -1: 'Missing'}))

Demo['education'] = pd.Categorical(Demo['education'].replace(
                    {1: 'Less than 9th grade', 
                    2: '9-11th grade (Includes 12th grade with no diploma)', 
                    3: 'High school graduate/GED or equivalent',
                    4: 'Some college or AA degree',
                    5: 'College graduate or above',
                    7: 'Refused',
                    9: 'Unknown',
                    -1: 'Missing'}))

Demo['marital_status'] = pd.Categorical(Demo['marital_status'].replace(
                        {1: 'Married', 
                        2: 'Widowed', 
                        3: 'Divorced',
                        4: 'Separated',
                        5: 'Never married',
                        6: 'Living with partner',
                        77: 'Refused',
                        99: 'Unknown',
                        -1: 'Missing'}))

Demo['interview_exam_status'] = pd.Categorical(Demo['interview_exam_status']
                                .replace(
                                {1: 'Interviewed only', 
                                2: 'Both interviewed and MEC examined', 
                                -1: 'Missing'}))

In [19]:
Demo.columns

Index(['id', 'age', 'gender', 'race', 'education', 'marital_status',
       'interview_exam_status', 'psu', 'stratum', 'two_year_mec_wt',
       'twoyr_interview_wt', 'cohort'],
      dtype='object')

In [20]:
# Change the rest of the float varaibels into integer variables 
fields_to_int = ['id', 'age', 'psu', 'stratum']
Demo[fields_to_int] = Demo[fields_to_int].applymap(np.int32)
Demo.to_pickle('./Demo_pickle.pkl')

The Denitition Dataset:

In [16]:
# Read the dataset 
df1112_dent = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2011-2012/OHXDEN_G.XPT')

/usr/local/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [18]:
# Specify the fields we are interested in
fields_dent = ['SEQN','OHDDESTS'] + \
            [ele for ele in df1112_dent.columns.values[4:64]]

In [19]:
df1112_dent = df1112_dent[fields_dent]
df1314_dent = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2013-2014/OHXDEN_H.XPT')[fields_dent]

In [20]:
df1516_dent = pd.read_sas(
      'https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/OHXDEN_I.XPT')[fields_dent]

In [21]:
df1718_dent = pd.read_sas(
    'https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/OHXDEN_J.XPT')[fields_dent]

In [22]:
# Add an indictor 'cohort' for the year of the sampling
df1112_dent['cohort'] = '2011-2012'
df1314_dent['cohort'] = '2013-2014'
df1516_dent['cohort'] = '2015-2016'
df1718_dent['cohort'] = '2017-2018'

In [23]:
Dent = df1112_dent.append(df1314_dent).append(df1516_dent).append(df1718_dent)

In [24]:
# Change the data column names to lower case 
Dent.columns = Dent.columns.str.lower()

In [25]:
# Create naming dictionaries for all data column 
d1 = {'seqn':'id','ohddests':'dentition status code'}
d2 = dict(zip(Dent.columns[2:34], 
    ['tooth count {}'.format(x+1) for x in range(len(Dent.columns[2:34]))]))

d3 = dict(zip(Dent.columns[34:48], 
    ['Coronal Caries Tooth Count {}'.
    format(x+2) for x in range(len(Dent.columns[34:48]))]))

d4 = dict(zip(Dent.columns[49:64], 
    ['Coronal Caries Tooth Count {}'.format(x) for x in range(19,32)]))
d1.update(d2)
d1.update(d3)
d1.update(d4)
# Rename the data columns
Dent = Dent.rename(d1,axis=1)

In [26]:
# Add category names for the categorical variables
Dent['dentition status code'] = pd.Categorical(Dent['dentition status code']
                            .replace({1: 'Complete', 
                            2: 'Partial', 3:'Not Done'}))
for i in range(2, 34):
    Dent[str(Dent.columns[i])] = pd.Categorical(Dent[str(Dent.columns[i])].
    replace(
        {1: 'Primary tooth (deciduous) present',
         2: 'Permanent tooth present', 
         3: 'Dental implant',
         4: 'Tooth not present', 
         5: 'Permanent dental root fragment present',
         9: 'Could not assess',
        -1: 'Missing'}))

for i in range(34, 62):
    Dent[str(Dent.columns[i])] = pd.Categorical(Dent[str(Dent.columns[i])].
    replace(
        {b'D': 'Sound primary tooth',
         b'E': 'Missing due to dental disease', 
         b'J': 'Permanent root tip is present but \
             no restorative replacement',
         b'K': 'Primary tooth with surface condition (s)', 
         b'M': 'Missing due to other causes',
         b'P': 'Missing due to dental disease but replaced \
             by a removable restoration',
         b'Q': 'Missing due to other causes but replaced by \
             a removable restoration',
         b'R': 'Missing due to dental disease, but replaced \
             by a fixed restoration',
         b'S': 'Sound permanent tooth',
         b'T': 'Permanent root tip is present but a restorative \
              replacement is present',
         b'U': 'Unerupted',
         b'X': 'Missing due to other causes, but replaced by \
              a fixed restoration',
         b'Y': 'Tooth present, condition cannot be assessed',
         b'Z': 'Permanent tooth with surface condition (s)',
         b'': 'Missing'}))

In [27]:
# Write to a pickle object
fields_to_int = ['id']
Dent[fields_to_int] = Dent[fields_to_int].applymap(np.int32)
Dent.to_pickle('./Dent_pickle.pkl')

In [28]:
print('The number of cases in the NHANES demographic ',
      'datasets from 2011 to 2018 is:', Demo['id'].nunique())
print('The number of cases in the NHANES oral health ', 
      'and dentition datasets from 2011 to 2018 is:', Dent['id'].nunique())

The number of cases in the NHANES demographic  datasets from 2011 to 2018 is: 39156
The number of cases in the NHANES oral health  and dentition datasets from 2011 to 2018 is: 35909
